In [4]:
# https://adrien.barbaresi.eu/blog/trafilatura-main-text-content-python.html

import requests
import bs4
from bs4 import BeautifulSoup
import os


# Get archives

In [5]:
page_url = "https://kghosh.substack.com/archive"

page_sourced = requests.get(page_url).content 

html_content = BeautifulSoup(page_sourced, "html.parser")
# #main > div.archive-page.typography.use-theme-bg > div > div > div.portable-archive-list > div:nth-child(19) > div.post-preview-content > a.post-preview-title.newsletter
links = html_content.findAll('a', class_="post-preview-title newsletter")
print(len(links),links) 
pages = [i.get('href') for i in links if (not (i.get('href') is None))]

pages = [ i for i in pages  if "https://kghosh.substack.com/p/" in i ]
pages = list(set([ i for i in pages  if not i.endswith("comments") ]))
len(pages), pages

12 [<a class="post-preview-title newsletter" href="https://kghosh.substack.com/p/20230521">20230521</a>, <a class="post-preview-title newsletter" href="https://kghosh.substack.com/p/20230513">20230513</a>, <a class="post-preview-title newsletter" href="https://kghosh.substack.com/p/20230505">20230505</a>, <a class="post-preview-title newsletter" href="https://kghosh.substack.com/p/20230423">20230423</a>, <a class="post-preview-title newsletter" href="https://kghosh.substack.com/p/20230416">20230416</a>, <a class="post-preview-title newsletter" href="https://kghosh.substack.com/p/fully-homomorphic-encryption">Fully Homomorphic Encryption</a>, <a class="post-preview-title newsletter" href="https://kghosh.substack.com/p/20230408">20230408</a>, <a class="post-preview-title newsletter" href="https://kghosh.substack.com/p/20230401">20230401</a>, <a class="post-preview-title newsletter" href="https://kghosh.substack.com/p/when-the-tide-of-ai-generated-texts">When the Tide of AI-generated Text

(12,
 ['https://kghosh.substack.com/p/20230312',
  'https://kghosh.substack.com/p/20230505',
  'https://kghosh.substack.com/p/20230521',
  'https://kghosh.substack.com/p/fully-homomorphic-encryption',
  'https://kghosh.substack.com/p/20230416',
  'https://kghosh.substack.com/p/20230408',
  'https://kghosh.substack.com/p/20230325',
  'https://kghosh.substack.com/p/20230423',
  'https://kghosh.substack.com/p/when-the-tide-of-ai-generated-texts',
  'https://kghosh.substack.com/p/20230513',
  'https://kghosh.substack.com/p/20230319',
  'https://kghosh.substack.com/p/20230401'])

In [6]:
for page in pages:
    name = page.split("/")[-1]

    if not os.path.exists(".cache/"+name):
        page_sourced = requests.get(page).content 
        html_content = BeautifulSoup(page_sourced, "html.parser")
        content = html_content.findAll('div', class_="body markup")

        with open(".cache/"+name, 'w') as f:
            f.write(str(content))
        print(name,"saved")
    else:
        print(name,"exists.")

20230312 exists.
20230505 exists.
20230521 saved
fully-homomorphic-encryption exists.
20230416 exists.
20230408 exists.
20230325 exists.
20230423 exists.
when-the-tide-of-ai-generated-texts exists.
20230513 exists.
20230319 exists.
20230401 exists.


In [7]:
import glob
import pandas as pd


cached_pages = glob.glob(".cache/*")


In [8]:
print("There are",len(cached_pages),'pages saved.')
cached_pages.sort()
cached_pages[:3]

There are 45 pages saved.


['.cache/20220118', '.cache/20220128', '.cache/20220212']

In [9]:
URLs = []
for page in cached_pages:
    with open(page) as fp:
        html_content = BeautifulSoup(fp, 'html.parser')  
    content = html_content.findAll('a')
    content = [i.get('href') for i in content if (not (i.get('href') is None))]
    for link in content:
        URLs.append([name,link])
    if "202305" in page:
        print(page,content) 

.cache/20230505 ['https://lareviewofbooks.org/article/the-flat-circle-on-jenny-odells-saving-time/?utm_source=substack&utm_medium=email', 'https://robwalker.substack.com/p/every-single-x?utm_source=substack&utm_medium=email', 'https://www.japantimes.co.jp/news/2023/04/20/national/chatgpt-yokosuka-trial/?utm_source=substack&utm_medium=email', 'https://www.atlasobscura.com/foods/mad-honey-red-hallucinogen?utm_source=Gastro%20Obscura%20Weekly%20E-mail&utm_campaign=8ce7161ff5-GASTRO_EMAIL_CAMPAIGN_2023_04_25&utm_medium=email&utm_term=0_2418498528-8ce7161ff5-76241438&mc_cid=8ce7161ff5&mc_eid=9fe71235bc', 'https://studyfinds.org/third-adults-sleep-comfort-object-childhood/?utm_source=substack&utm_medium=email', 'https://psyche.co/guides/how-to-wander-in-a-world-that-values-purpose?utm_source=substack&utm_medium=email', 'https://www.wired.com/story/an-ominous-heating-event-is-unfolding-in-the-oceans/', 'https://consequence.net/2023/04/half-vinyl-buyers-record-player-study/', 'https://www.salo

In [10]:
import hashlib

df = pd.DataFrame(URLs)
df.columns = ["page","url"]
df = df.drop_duplicates(subset=["url"])
df["url"] = df.url.apply(lambda x: str(x).encode('utf-8'))

df["hash"] = df.url.apply(lambda x: hashlib.md5(str(x).encode('utf-8')).hexdigest())
df.to_parquet("urls.parquet.gzip",  engine='pyarrow', compression='gzip')
df

,page,url,hash
0,20230401,b'https://www.gov.wales/sites/default/files/pu...,3b92dc627d73b0d586fcff194b697c89
1,20230401,b'https://www.theguardian.com/world/2023/feb/1...,b3b0e79abac9ac8ba36e389c1d09621d
2,20230401,b'https://twitter.com/TrungTPhan/status/162664...,5d265f36c41b6d01ca52ab0f99ee8e95
3,20230401,b'https://www.futuregenerations.wales/about-us...,3938700a79bbde32972657f9eb7b1b69
4,20230401,b'https://www.bbc.com/future/article/20230215-...,32228bbf929d81d39a1808dd9b7c4493
...,...,...,...
1049,20230401,b'https://web.yammer.com/main/org/mottmac.com/...,fc70437f7665556e8d27c5a9653763a6
1050,20230401,b'https://www.kaggle.com/code/concretemlteam/t...,cde046296faf786f02951f73b9b25417
1051,20230401,"b'https://substackcdn.com/image/fetch/f_auto,q...",f0a23dcd98c2fdf0a3fd523a04e7c255
1052,20230401,"b'https://substackcdn.com/image/fetch/f_auto,q...",d256bfc733bfe7fe3f55726967800605
